#Determinação Parâmetros - Motores de Indução **TRIFÁSICOS**



In [ ]:
import math
import numpy as np
import cmath as cm

"""
Dados do Motor
"""

V_nom = 400 #V
P = 6
f_e = 50 #Hz
n_sinc = 1000 #rpm
enrol = "triangulo"
f_op = 50 # frequencia de operação, Hz
cat_rot = "A"

n_mec = 0
s = 0.075

Prot = 1250 #W
V_op = 400 #operação LINHA (V)

if P>0:
  n_sinc = (120/P)*f_e
else:
  n_sinc = n_sinc

if n_mec > 0:
  s = 1 - n_mec/n_sinc
if n_mec == 0:
  n_mec = (1-s)*n_sinc

if enrol == "estrela":
  V_op_fase = V_op/math.sqrt(3)
if enrol == "triangulo":
  V_op_fase = V_op




"""
Dados dos Ensaios
"""
#VAZIO
V_v = 400 #V
I_v = 8 #A
P_v = 280 #W
f_v = 50 # Hz

#ROTOR BLOQ.
V_rb = 140 #V
I_rb = 38 #A
P_rb = 2500 #W
f_rb = 12.5 #Hz

#Resistência
R1 = 0.6 #Ohms

"""
Cálculos Vazio
"""

if enrol == "estrela":
  V_vf = V_v/math.sqrt(3)
  I_vf = I_v
if enrol == "triangulo":
  V_vf = V_v
  I_vf = I_v/math.sqrt(3)

Z_v = V_vf/I_vf
R_v = P_v/(3*(I_vf**2))
X_v = math.sqrt(Z_v**2 - R_v**2)

"""
Cálculos rotor bloq.
"""

if enrol == "estrela":
  V_rb_f = V_rb/math.sqrt(3)
  I_rb_f = I_rb
if enrol == "triangulo":
  V_rb_f = V_rb
  I_rb_f = I_rb/math.sqrt(3)

Z_rb = V_rb_f/I_rb_f
R_rb = P_rb/(3*(I_rb_f**2))
X_rb = math.sqrt(Z_rb**2 - R_rb**2)

X_rb_freq = (f_op/f_rb)*X_rb

"""
Determinação X1 e X2
"""

if cat_rot in ["A", "D", "Bobinado"]:
  X_1 = 0.5*X_rb_freq
  X_2 = 0.5*X_rb_freq
if cat_rot == "B":
  X_1 = 0.4*X_rb_freq
  X_2 = 0.6*X_rb_freq
if cat_rot == "C":
  X_1 = 0.3*X_rb_freq
  X_2 = 0.7*X_rb_freq
if cat_rot in ["N", "H", "D_e_bob"]:
  if cat_rot == "N":
    k = 0.68
  if cat_rot == "H":
    k = 0.58
  if cat_rot == "0.78":
    k = 0.78
  A = np.matrix([[1, k], [1, 1]])
  B = np.array([0, X_rb_freq])
  x = np.linalg.solve(A, B)
  x[0] = X_1
  x[1] = X_2


"""
Determinação de Xm
"""

X_m = X_v - X_1


"""
Determinação de R2
"""

R2 = (R_rb - R1)*((X_2 + X_m)**2)/(X_m**2)

"""
Correntes
"""
#Zm
Rc = 0
j = cm.sqrt(-1)

if Rc == 0:
  Zm = (X_m)*j
else:
  Zm = (j*X_m)/(1+j*X_m/Rc)

#Z1 E Z2
Z1 = R1 + j*X_1
Z2 = R2/s + j*X_2

#Zeq
Zeq = ((Z2**-1) + (Zm**-1))**-1 

#I1 e I2
I1 = V_op_fase/(R1 + j*X_1 + Zeq)
I2 = I1*(Zm/(Zm+Z2))


"""
Potência e Conjugado
"""
I1_abs = cm.polar(I1)[0]
I1_fase = cm.polar(I1)[1]
I2_abs = cm.polar(I2)[0]
fp = math.cos(I1_fase)

Pce = 3*(I1_abs**2)*R1
PEF = 3*(I2_abs**2)*R2/s
Pconv = (1-s)*PEF
Pout = Pconv - Prot
Pin = 3*V_op_fase*I1_abs*fp

omega_sinc = (math.pi/30)*n_sinc
omega_mec = (math.pi/30)*n_mec

C_ind = PEF/omega_sinc
C_eixo = Pout/omega_mec

eta = 100*(Pout/Pin)


"""
Resultados
"""

print(f"Tensão Nominal: {V_nom} V")
print(f"Veloc. síncrona: {n_sinc} rpm")
print(f"Enrolamento: {enrol}")
print(f"Freq. de Operação: {f_op} Hz")
print(f"Categoria do Rotor: {cat_rot}")
print(f"Tensão de operação (linha): {V_op} V")

print(f"\nZ_v = {round(Z_v,3)} Ohms")
print(f"R_v = {round(R_v,3)} Ohms")
print(f"X_v = {round(X_v,3)} Ohms")
print(f"Z_rb = {round(Z_rb,3)} Ohms")
print(f"R_rb = {round(R_rb,3)} Ohms")
print(f"X_rb = {round(X_rb,3)} Ohms")

print(f"\nR1 = {round(R1,3)} Ohms \nX_1 = {round(X_1,3)} Ohms")
print(f"R2 = {round(R2,3)} Ohms \nX_2 = {round(X_2, 3)} Ohms \nX_m = {round(X_m,3)} Ohms")

print(f"\nI1 = {round(cm.polar(I1)[0], 3)} /_ {round(math.degrees(cm.polar(I1)[1]),3)}º")
print(f"I2 = {round(cm.polar(I2)[0], 3)} /_ {round(math.degrees(cm.polar(I2)[1]),3)}º")

print(f"Conjugado induzido: {round(C_ind,3)} Nm")
print(f"Conjugado no eixo: {round(C_eixo,3)} Nm")
print(f"Eficiência: {round(eta,3)} %")
print(f"Fator de Potência: {round(fp,3)}")

Tensão Nominal: 400 V
Veloc. síncrona: 1000.0 rpm
Enrolamento: triangulo
Freq. de Operação: 50 Hz
Categoria do Rotor: A
Tensão de operação (linha): 400 V

Z_v = 86.603 Ohms
R_v = 4.375 Ohms
X_v = 86.492 Ohms
Z_rb = 6.381 Ohms
R_rb = 1.731 Ohms
X_rb = 6.142 Ohms

R1 = 0.6 Ohms 
X_1 = 12.284 Ohms
R2 = 1.537 Ohms 
X_2 = 12.284 Ohms 
X_m = 74.208 Ohms

I1 = 13.272 /_ -60.408º
I2 = 11.081 /_ -47.08º
Conjugado induzido: 72.077 Nm
Conjugado no eixo: 59.172 Nm
Eficiência: 72.877 %
Fator de Potência: 0.494


#Determinação Parâmetros - Motores de Ind. **MONOFÁSICOS**

In [ ]:
import math
import numpy as np
import cmath as cm

"""
Dados do Motor
"""

V_nom = 115 #V
P = 6
f_e = 60 #Hz
n_sinc = 0 #rpm
enrol = "triangulo"
f_op = 50 # frequencia de operação, Hz

n_mec = 1120
s = 0
Prot = 0 #W (QUANDO NÃO HÁ INFO)
V_op = 115 #operação 

if P>0:
  n_sinc = (120/P)*f_e
else:
  n_sinc = n_sinc

if n_mec > 0:
  s = 1 - n_mec/n_sinc
if n_mec == 0:
  n_mec = (1-s)*n_sinc



"""
Dados dos Ensaios
"""
#VAZIO
V_v = 115 #V
I_v = 3.2 #A
P_v = 55.2 #W


#ROTOR BLOQ.
V_rb = 25 #V
I_rb = 3.7 #A
P_rb = 86.2 #W


#Resistência ESTATOR
R1 = 2.5 #Ohms

"""
Cálculos Vazio
"""

Z_v = V_v/I_v
R_v = P_v/(I_v**2)
X_v = math.sqrt(Z_v**2 - R_v**2)

"""
Cálculos rotor bloq.
"""

Z_rb = V_rb/I_rb
R_rb = P_rb/(I_rb**2)
X_rb = math.sqrt(Z_rb**2 - R_rb**2)

"""
Determinação X1 e X2
"""

X_1 = 0.5*X_rb
X_2 = X_1


"""
Determinação de Xm
"""

X_m = 2*X_v - 1.5*X_1


"""
Determinação de R2
"""

R2 = (R_rb - R1)

"""
Corrente I1
"""
#Zprog e Zretr
j = cm.sqrt(-1)

Zprog = (((R2/s)+j*X_2)*(j*X_m))/(((R2/s)+j*X_2)+(j*X_m))
Zretr = (((R2/(2-s))+j*X_2)*(j*X_m))/(((R2/(2-s))+j*X_2)+(j*X_m))

#I1
I1 = V_op/(R1 + j*X_1 + 0.5*(Zprog + Zretr))


"""
Potência e Conjugado
"""
I1_abs = cm.polar(I1)[0]
I1_fase = cm.polar(I1)[1]
fp = math.cos(I1_fase)

Pef_prog = (I1_abs**2)*(0.5*Zprog.real)
Pef_retr = (I1_abs**2)*(0.5*Zretr.real)
PEF = Pef_prog - Pef_retr

Pconv = (1-s)*PEF

if Prot == 0:
  Prot = P_v - (I_v**2)*R1

Pout = Pconv - Prot
Pin = V_op*I1_abs*fp

omega_sinc = (math.pi/30)*n_sinc
omega_mec = (math.pi/30)*n_mec

C_ind = PEF/omega_sinc
C_eixo = Pout/omega_mec

eta = 100*(Pout/Pin)


"""
Resultados
"""

print(f"Tensão Nominal: {V_nom} V")
print(f"Veloc. síncrona: {n_sinc} rpm")
print(f"Tensão de operação: {V_op} V")

print(f"\nZ_v = {round(Z_v,3)} Ohms")
print(f"R_v = {round(R_v,3)} Ohms")
print(f"X_v = {round(X_v,3)} Ohms")
print(f"Z_rb = {round(Z_rb,3)} Ohms")
print(f"R_rb = {round(R_rb,3)} Ohms")
print(f"X_rb = {round(X_rb,3)} Ohms")

print(f"\nR1 = {round(R1,3)} Ohms \nX_1 = {round(X_1,3)} Ohms")
print(f"R2 = {round(R2,3)} Ohms \nX_2 = {round(X_2, 3)} Ohms \nX_m = {round(X_m,3)} Ohms")

print(f"\nI1 = {round(cm.polar(I1)[0], 3)} /_ {round(math.degrees(cm.polar(I1)[1]),3)}º")

print(f"Conjugado induzido: {round(C_ind,3)} Nm")
print(f"Conjugado no eixo: {round(C_eixo,3)} Nm")
print(f"Eficiência: {round(eta,3)} %")
print(f"Fator de Potência: {round(fp,3)}")
print(f"Pout = {round(Pout, 3)} W")
print(f"Pin = {round(Pin, 3)} W")

Tensão Nominal: 115 V
Veloc. síncrona: 1200.0 rpm
Tensão de operação: 115 V

Z_v = 35.938 Ohms
R_v = 5.391 Ohms
X_v = 35.531 Ohms
Z_rb = 6.757 Ohms
R_rb = 6.297 Ohms
X_rb = 2.451 Ohms

R1 = 2.5 Ohms 
X_1 = 1.225 Ohms
R2 = 3.797 Ohms 
X_2 = 1.225 Ohms 
X_m = 69.224 Ohms

I1 = 4.491 /_ -38.375º
Conjugado induzido: 2.517 Nm
Conjugado no eixo: 2.264 Nm
Eficiência: 65.591 %
Fator de Potência: 0.784
Pout = 265.572 W
Pin = 404.892 W


#CKT Motores de Ind. **MONOFÁSICOS**

In [ ]:
import math
import numpy as np
import cmath as cm

"""
Dados do Motor
"""

V_nom = 110 #V
P = 4
f_e = 60 #Hz
n_sinc = 0 #rpm
f_op = 60 # frequencia de operação, Hz

n_mec = 1760
s = 0.04
Prot = 58 #W (QUANDO NÃO HÁ INFO)
V_op = 110 #operação 

if P>0:
  n_sinc = (120/P)*f_e
else:
  n_sinc = n_sinc

if n_mec > 0:
  s = 1 - n_mec/n_sinc
if n_mec == 0:
  n_mec = (1-s)*n_sinc



"""
Parâmetros
"""

R1 = 1.2 #Ohms
R2 = 2.5 #Ohms
X_1 = 2.4 #Ohms
X_2 = 2.4 #Ohms
X_m = 74 #Ohms

"""
Corrente I1
"""
#Zprog e Zretr
j = cm.sqrt(-1)

Zprog = (((R2/s)+j*X_2)*(j*X_m))/(((R2/s)+j*X_2)+(j*X_m))
Zretr = (((R2/(2-s))+j*X_2)*(j*X_m))/(((R2/(2-s))+j*X_2)+(j*X_m))

#I1
I1 = V_op/(R1 + j*X_1 + 0.5*(Zprog + Zretr))


"""
Potência e Conjugado
"""
I1_abs = cm.polar(I1)[0]
I1_fase = cm.polar(I1)[1]
fp = math.cos(I1_fase)

Pef_prog = (I1_abs**2)*(0.5*Zprog.real)
Pef_retr = (I1_abs**2)*(0.5*Zretr.real)
PEF = Pef_prog - Pef_retr

Pconv = (1-s)*PEF

if Prot == 0:
  Prot = P_v - (I_v**2)*R1

Pout = Pconv - Prot
Pin = V_op*I1_abs*fp

omega_sinc = (math.pi/30)*n_sinc
omega_mec = (math.pi/30)*n_mec

C_ind = PEF/omega_sinc
C_eixo = Pout/omega_mec

eta = 100*(Pout/Pin)


"""
Resultados
"""

print(f"Tensão Nominal: {V_nom} V")
print(f"Veloc. síncrona: {n_sinc} rpm")
print(f"Tensão de operação: {V_op} V")


print(f"\nR1 = {round(R1,3)} Ohms \nX_1 = {round(X_1,3)} Ohms")
print(f"R2 = {round(R2,3)} Ohms \nX_2 = {round(X_2, 3)} Ohms \nX_m = {round(X_m,3)} Ohms")

print(f"\nZprog = {round(np.real(Zprog),3)} + j{round(np.imag(Zprog),3)} ")
print(f"Zretr = {round(np.real(Zretr),3)} + j{round(np.imag(Zretr),3)} ")

print(f"\nI1 = {round(cm.polar(I1)[0], 3)} /_ {round(math.degrees(cm.polar(I1)[1]),3)}º")
print(f"Fator de Potência: {round(fp,3)}")
print(f"Pout = {round(Pout, 3)} W")
print(f"Pin = {round(Pin, 3)} W")

print(f"\nConjugado induzido: {round(C_ind,3)} Nm")
print(f"Conjugado no eixo: {round(C_eixo,3)} Nm")
print(f"Eficiência: {round(eta,3)} %")


Tensão Nominal: 110 V
Veloc. síncrona: 1800.0 rpm
Tensão de operação: 110 V

R1 = 1.2 Ohms 
X_1 = 2.4 Ohms
R2 = 2.5 Ohms 
X_2 = 2.4 Ohms 
X_m = 74 Ohms

Zprog = 33.312 + j51.377 
Zretr = 1.186 + j2.344 

I1 = 3.18 /_ -57.769º
Fator de Potência: 0.533
Pout = 100.829 W
Pin = 186.562 W

Conjugado induzido: 0.862 Nm
Conjugado no eixo: 0.547 Nm
Eficiência: 54.046 %
